Modele Simple: Génération des embeddings pour Designation

In [15]:
import importlib


from RktnChallenge.RktnModel import ModelTrainer
from RktnChallenge.preprocessing.tokenizeString import tokenizeString
from RktnChallenge.preprocessing.filterStopWords import filterStopWords
from RktnChallenge.preprocessing.mergeFeatures import mergeFeatures
from RktnChallenge.preprocessing.mostOccur import mostOccur
from RktnChallenge.preprocessing.Dropper import Dropper
from RktnChallenge.preprocessing.TokenListToString import TokenListToString

In [16]:
RktnModel = ModelTrainer("data_tr_train.csv")

Vectorization des données

In [18]:
_X = RktnModel.data["merged_desi_desc"]

RktnModel.create_vectorizer(name="tokenizer", X = _X, ngram_range=(1,3))

X_data = RktnModel.vectorizer_transform(_X)     


y = RktnModel.encodeLabel("prdtypecode")
label_size = RktnModel.getLabelSize()

import json
#on sauve le vectorizer
tokenizer_json = RktnModel.tokenizer.to_json()
with open('desi_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

import joblib
joblib.dump(RktnModel.encoder, 'desi_encoder.pkl')

['desi_encoder.pkl']

In [19]:
from sklearn.model_selection import train_test_split

X_train = X_data
y_train = y
print(RktnModel.max_seq_length)

47


Definition d'un modele

In [8]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRUCell,RNN,GRU, Dense,Dropout,GlobalAveragePooling2D, LSTM, Flatten, Input
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.models import Model

_latent_dim = 150

vocab_size = RktnModel.vocab_size
max_seq_length = RktnModel.max_seq_length
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=_latent_dim, input_length=max_seq_length, mask_zero=True),     
    Flatten(),   
    #Dense(units=nb_units, activation="relu"),
    Dense(units=label_size, activation="softmax")
])


 
optimizer = AdamW(weight_decay=0.01, learning_rate=0.001) 
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 47, 150)           10582650  
                                                                 
 flatten_1 (Flatten)         (None, 7050)              0         
                                                                 
 dense_1 (Dense)             (None, 27)                190377    


                                                                 
Total params: 10773027 (41.10 MB)
Trainable params: 10773027 (41.10 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
history = model.fit(X_train, y_train, epochs=4, batch_size=100, validation_split=0.2)

Epoch 1/4
544/544 [==============================] - 22s 40ms/step - loss: 1.5857 - accuracy: 0.5803 - val_loss: 0.8273 - val_accuracy: 0.7713
Epoch 2/4
544/544 [==============================] - 7s 13ms/step - loss: 0.5084 - accuracy: 0.8656 - val_loss: 0.6360 - val_accuracy: 0.8103
Epoch 3/4
544/544 [==============================] - 6s 10ms/step - loss: 0.2180 - accuracy: 0.9494 - val_loss: 0.6139 - val_accuracy: 0.8139
Epoch 4/4
544/544 [==============================] - 4s 8ms/step - loss: 0.1005 - accuracy: 0.9788 - val_loss: 0.6253 - val_accuracy: 0.8112


In [10]:



model.save("model_designation.h5")

/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
